# Importance of features example with scikit-learn

This file serves as simple example of how to use some of scikit-learn capabilities for feature selection and how to chain various algorithms for classification/regression.

First a group of decision trees is used in order to determine how it splits the trees according to an entropy criterion and establishing some form of importance in the features.

The second example first uses PCA to transform the representation of the features and then passes the new instances to a regressor (logistic regression/decision tree). Then through grid search it is estimated how many components should the model take.

[Decision tree criterions for relative importance](#A1)

[PCA and regressor pipeline](#A2)

[Component selection](#A3)

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rcParams
import pandas as pd
import warnings

from scipy.stats import beta

from ipywidgets import widgets
from IPython.html.widgets import *

warnings.filterwarnings('ignore')
rcParams.update({'font.size': 15})
#plt.style.use('ggplot')
#plt.style.use('seaborn-dark-palette')
plt.style.use('fivethirtyeight')

float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
np.set_printoptions(precision=2)


import io
from IPython.nbformat import current

def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)

/usr/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
execute_notebook("Preprocessing.ipynb")

['free']
250
['free', 'train']
['free', 'test', 'train']


In [3]:
tasks = 4
catLab = ['1D', 'I1D', '2D', 'R']

usersF = np.shape(np.unique(csvIntF[:,0]))[0]#5
# For free exploration with Training
usersFT = np.shape(np.unique(csvIntFT[:,0]))[0]
usersFTI = np.shape(np.unique(informed[:,0]))[0]
usersFTU = usersFT-usersFTI

#### Additional Preprocessing

In [4]:
# Load behavioral trajectories (csvIntFT)
# Get only those in training phase
csvIntFTT = csvIntFT[csvIntFT[:,2]==1]
# Get a copy for splitting by condition (below)
csvIntFTTCond = csvIntFTT.copy()

# Remove phase column (2) (and for now also the condition column (1))
csvIntFTT = np.delete(csvIntFTT, (1,2), axis=1)

# Split by condition
# Get rid of phase column
csvIntFTTCond = np.delete(csvIntFTTCond, 2, axis = 1)
# Split by informed/uninformed
informedFTT = csvIntFTTCond[csvIntFTTCond[:,1]==0]
uninformedFTT = csvIntFTTCond[csvIntFTTCond[:,1]==1]
informedFTT = np.delete(informedFTT, 1, axis=1)
uninformedFTT = np.delete(uninformedFTT, 1, axis=1)
# Split by user
splitInfFTT = [informedFTT[informedFTT[:,0]==i] for i in np.unique(informedFTT[:,0])]
splitUniFTT = [uninformedFTT[uninformedFTT[:,0]==i] for i in np.unique(uninformedFTT[:,0])]

#### Self-reported answers preprocessing

In [5]:
execute_notebook("Preprocessing-Stack Data.ipynb")

Free Exploration - general metrics loaded
Free Exploration with Training - general metrics loaded
Strategic Learning - general metrics loaded


In [6]:
#user(0), cond(1) cat-task complexity(2), # task selec(3), % sele(4), # correct on task(5), % correct(6), 
#answers(7:12)

# Split by users that received information about the existence of a random task and those that didn't
informedFT = freeT[freeT[:,1]==0]
uninformedFT = freeT[freeT[:,1]==1]
# For training only
informedFTT = freeTTr[freeTTr[:,1]==0]
uninformedFTT = freeTTr[freeTTr[:,1]==1]

#print(spilot[-1,:])
# Remove column 
freeT = np.delete(freeT, 1, axis=1)
informedFT = np.delete(informedFT, 1, axis=1)
uninformedFT = np.delete(uninformedFT, 1, axis=1)
informedFTT = np.delete(informedFTT, 1, axis=1)
uninformedFTT = np.delete(uninformedFTT, 1, axis=1)
free = np.delete(free, 1, axis=1)

#### Rate of empirical success

In [7]:
# Rate of success
# To use as alternative to the probabilities
TRIALS_TRAINING = 15
def rateSucc(arr, userArr):
    # Store performance per user per task
    perfUser = []
    # Go through user
    for u in range(userArr):
        tmpU = arr[u]
        perfTask = []
        # Go through each task
        for t in range(tasks):
            # Split by task
            tmpT = tmpU[tmpU[:,1] == t]
            perfTask.append(np.sum(tmpT[:,2]==1)/TRIALS_TRAINING)
        perfUser.append(perfTask)
            
    return np.asarray(perfUser)

def relRateSucc(arr):
    return arr/np.mean(arr, axis=1).reshape(np.shape(arr)[0],1)

# Obtain predicted accuracy (e.g. "I predict based on my observations that my p(correct)=? if you ask me 
# to classify instances of this exercise without receiving any feedback")
rateFTTI = rateSucc(splitInfFTT, usersFTI)
rateFTTU = rateSucc(splitUniFTT, usersFTU)

relRateFTTI = relRateSucc(rateFTTI)
relRateFTTU = relRateSucc(rateFTTU)

#user(0), cat-task complexity(1), # task selec(2), % sele(3), # correct on task(4), % correct(5), 
#answers(6:12), relative performance
informedFTT = np.column_stack((informedFTT, relRateFTTI.flatten()))

ValueError: all the input array dimensions except for the concatenation axis must match exactly

#### Probability success trajectories

In [8]:
# Gets the p(correct) at each trial per task and per user
def extractProbsTrajec(arr, userArr):
    # Store p(correct) per user
    probUser = []
    # Go through user
    for u in range(userArr):
        tmpU = arr[u]
        # Init prior
        alpha,bet = 2,2
        # Store p(correct) = predicted accuracy evolution per task for user u
        probTask = []
        # Go through each task
        for t in range(tasks):
            # Split by task
            tmpT = tmpU[tmpU[:,1] == t]
            # Go through its trial history
            succ = 0
            # store p(correct) in this task
            probs = []
            probs.append(np.mean(beta(alpha+succ, bet+(0+1)-succ).rvs(size=500)))
            #print(np.mean(beta(alpha+succ, bet+(0+1)-succ).rvs(size=500)), alpha+succ, bet+0-succ)
            for trial in range(np.shape(tmpT)[0]):
                # Check if the answer was correct on this trial
                if tmpT[trial,2] == 1:
                    succ+=1
                #Construct beta distribution for posterior Beta(α=prior α+Succ, β=prior β+Trials−Succ)
                dist = beta(alpha+succ, bet+(trial+1)-succ)
                #Draw sample from beta distribution
                #print(np.mean(dist.rvs(size=500)), alpha+succ, bet+(trial+1)-succ)
                probs.append(np.mean(dist.rvs(size=500)))
            probTask.append(probs)
        probUser.append(probTask)
    return np.asarray(probUser)

# Obtain predicted accuracy (e.g. "I predict based on my observations that my p(correct)=? if you ask me 
# to classify instances of this exercise without receiving any feedback")
probsFTTI = extractProbsTrajec(splitInfFTT, usersFTI)
probsFTTU = extractProbsTrajec(splitUniFTT, usersFTU)

#### Alternatively use empirical rate of success trajectory (probably noisier during the first trials)

In [9]:
def extractSuccTrajec(arr, userArr):
    # Store rate of success per user
    succUser = []
    # Go through user
    for u in range(userArr):
        tmpU = arr[u]
        # Store evolution rate of success per task
        succTask = []
        # Go through each task
        for t in range(tasks):
            # Split by task
            tmpT = tmpU[tmpU[:,1] == t]
            # Go through its trial history
            succ = 0
            # store rates of success in this task
            rates = []
            # Added initial 0 so when transformed to error it starts from 1. However this value doesn't affect rate 
            # of success
            rates.append(succ)
            for trial in range(np.shape(tmpT)[0]):
                # Check if the answer was correct on this trial
                if tmpT[trial,2] == 1:
                    succ+=1
                rates.append(succ / (trial+1) )
            succTask.append(rates)
        succUser.append(succTask)
    return np.asarray(succUser)

succFTTI = extractSuccTrajec(splitInfFTT, usersFTI)
succFTTU = extractSuccTrajec(splitUniFTT, usersFTU)

#### Learning progress

In [10]:
def getLPNormal(rates, userArr):
    lpsUser = []
    # Go through every user and task
    for u in range(userArr):
        #tmpU = diff[u]
        tmpU = rates[u]
        lps = []
        for t in range(tasks):
            # Convert prob to errors
            errT = 1-tmpU[t]
            # fit polinomial to error differences
            slope, intercept = np.polyfit(np.arange(len(errT)), errT, 1)
            x = np.linspace(0, len(errT)-1, 100)
            # LP = - [Fitted Error(present) - Fitted Error(past)]
            lp = -((slope*x[-1]+intercept)-(slope*x[0]+intercept))
            lps.append(lp)
            #print(slope*x[0]+intercept, slope*x[-1]+intercept, diffT, lp)
            #print(probs[u], errT, diffT, slope*x[0]+intercept, slope*x[-1]+intercept, lp)
        lpsUser.append(lps)
        
        #print(lpsUser)
    return np.asarray(lpsUser)

def relLP(arr):
    return arr/np.mean(arr, axis=1).reshape(np.shape(arr)[0],1)

# Get LP
lpFTTI_N = getLPNormal(succFTTI, usersFTI)
lpFTTU_N = getLPNormal(succFTTU, usersFTU)

lpRelFTTI_N = relLP(lpFTTI_N)
lpRelFTTU_N = relLP(lpFTTU_N)

#user(0), cat-task complexity(1), # task selec(2), % sele(3), # correct on task(4), % correct(5), 
#answers(6:12), relative performance, lp, relative lp
informedFTT = np.column_stack((informedFTT, lpFTTI_N.flatten(), lpRelFTTI_N.flatten()))

ValueError: all the input array dimensions except for the concatenation axis must match exactly

#### Check first selection

In [12]:
def checkFirstOption(arr, usersArr, checkIncomplete=False):
    firstSelec = []
    incomplete = []
    for u in range(usersArr):
        # If it's necessary to check those that didn't explore all tasks (e.g. free exploration only)
        if checkIncomplete:
            questions = arr[u]
            # Count number of times a task was selected
            task, ctask = np.unique(questions[:,1], return_counts=True)

            # Check if it explored all tasks
            if len(task) < 4:
                incomplete.append(u)
            else:
                firstSelec.append(arr[u][0,1])
        else:
            firstSelec.append(arr[u][0,1])
            
    # Return also a list of people who didn't explore all tasks
    return np.asarray(firstSelec), incomplete

fsFTI,_ = checkFirstOption(splitCsvFTI, usersFTI)
fsFTU,_ = checkFirstOption(splitCsvFTU, usersFTU)

In [13]:
# Label is whether or not this task was the first option
def addLabel(arr, firstSelec):
    fsList = []
    for i, ii in enumerate(np.arange(0,np.shape(arr)[0],4)):
        sliced = arr[ii:ii+4]
        fs = sliced[:,1] == firstSelec[i]
        fs = fs.astype(int)
        fsList.append(fs.tolist())
    return np.asarray(fsList)

labelFTTI = addLabel(informedFTT, fsFTI)
informedFTT = np.column_stack((informedFTT, labelFTTI.flatten()))

#user(0), cat-task complexity(1), # task selec(2), % sele(3), # correct on task(4), % correct(5), 
#answers(6:12), relative performance, lp, relative lp
informedFTT[0:4]
# Randomize
np.take(informedFTT, np.random.permutation(informedFTT.shape[0]), axis=0, out=informedFTT)
# Remove column of user id
informedFTT = np.delete(informedFTT, 0, 1)

<a id='A1'></a>
## Decision tree criterions for relavite importance

In [14]:
from sklearn.model_selection import KFold
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier


# K-fold CV
# FOLDS = 3
# kf = KFold(n_splits=FOLDS) # Define the split - into 2 folds 
# kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
# KFold(n_splits=2, random_state=None, shuffle=True)

X, Y = informedFTT[:,:-1], informedFTT[:,-1] 
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)


# Decision tree
classifier = DecisionTreeClassifier(criterion = "entropy", random_state = 100, max_depth=3, min_samples_leaf=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print("Accuracy is ", accuracy_score(y_test,y_pred)*100)

# fit an Extra Trees model to the data
model = ExtraTreesClassifier(criterion = "entropy")
model.fit(X_train, y_train)
# Relative importance of each attribute
features = ['Complexity', 'Task selection', '% Task selection', '# Correct', '% Correct', 'Reported Interest',
           'Reported Complexity', 'Reported Time Invested', 'Reported Progress', 'Reported Rule',
            'Reported Future Learning after training', 'Reported Future Learning after experiment',
           'Relative Performance', 'LP', 'Relative LP']

print("==========")
zipped = zip(features, model.feature_importances_)
zipped = sorted(zipped, key=lambda x: x[1], reverse=True)
for f,i in zipped:
    print(f," ",i)

ImportError: No module named model_selection

<a id='A2'></a>
## PCA and regressor pipeline

In [15]:
# PCA + Logistic regression or Decision tree
from sklearn import linear_model, decomposition
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale

X_train_scaled = scale(X_train)

#classifier = linear_model.LogisticRegression()
classifier = DecisionTreeClassifier(criterion = "entropy", random_state = 100, max_depth=3, min_samples_leaf=5)

pca = decomposition.PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', classifier)])
pca.fit(X_train_scaled)

#X_proj = pca.transform(X_train_scaled)

#The amount of variance that each component explains
var = pca.explained_variance_ratio_

#Cumulative explained variance
varcum = np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

plt.plot(varcum)
plt.axis('tight')
plt.xlabel('# of components')
plt.ylabel('Cumulative Explained Variance')

ImportError: No module named model_selection

<a id='A3'></a>
## Component Selection

In [16]:
nComp = [5, 6, 7, 8, 9, 10, 11, 12]

estimator = GridSearchCV(pipe,dict(pca__n_components=nComp))
estimator.fit(X_train, y_train)

plt.axvline(estimator.best_estimator_.named_steps['pca'].n_components,linestyle='-', label='Components chosen')
plt.legend(prop=dict(size=12))
plt.show()

NameError: name 'GridSearchCV' is not defined